In [1]:
# bring in our LLAMA_CLOUD_API_KEY
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import nest_asyncio
import asyncio
from llama_parse import LlamaParse
from llama_index.core import SimpleDirectoryReader

In [3]:
# Apply nest_asyncio to allow async calls in Jupyter
nest_asyncio.apply()

In [4]:
# Set up parser
parser = LlamaParse(result_type="markdown")

In [5]:
# Create an async function to call aload_data explicitly
async def load_documents_async():
    file_extractor = {".pdf": parser}
    documents = SimpleDirectoryReader(input_files=['../data/layout-parser-paper.pdf'], file_extractor=file_extractor)
    # Explicitly call async load function if available
    if hasattr(documents, "aload_data"):
        return await documents.aload_data()
    else:
        return documents.load_data()  # Fallback if only a synchronous load is available

In [6]:
# Run the async function using the Jupyter-compatible event loop
documents = asyncio.run(load_documents_async())
print(documents)

Started parsing the file under job_id 660bbeff-8dcf-4545-86d0-e88add47dcfd
[Document(id_='0761496f-c807-4c9c-b168-24fe8845b15e', embedding=None, metadata={'file_path': '..\\data\\layout-parser-paper.pdf', 'file_name': 'layout-parser-paper.pdf', 'file_type': 'application/pdf', 'file_size': 4686220, 'creation_date': '2024-10-28', 'last_modified_date': '2024-10-26'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='# LayoutParser: A Unified Toolkit for Deep Learning Based Document Image Analysis\n\nZejiang Shen1, Ruochen Zhang2, Melissa Dell3, Benjamin Charles Germain Lee4, Jacob Carlson3, and Weining Li5\n\n1 Allen Institute for AI, shannons@allenai.org\n\n2 Brown University, ruochen_zhang@brown.edu\n\n3 Harvard University, {melissadell,jacob_carlson}@fas.h

In [7]:
len(documents)

16

In [8]:
# Print each document's content individually
for i, doc in enumerate(documents):
    print(f"Document {i + 1}:\n{doc}\n{'-' * 80}\n")


Document 1:
Doc ID: 0761496f-c807-4c9c-b168-24fe8845b15e
Text: # LayoutParser: A Unified Toolkit for Deep Learning Based
Document Image Analysis  Zejiang Shen1, Ruochen Zhang2, Melissa Dell3,
Benjamin Charles Germain Lee4, Jacob Carlson3, and Weining Li5  1
Allen Institute for AI, shannons@allenai.org  2 Brown University,
ruochen_zhang@brown.edu  3 Harvard University,
{melissadell,jacob_carlson}@fas.harvard...
--------------------------------------------------------------------------------

Document 2:
Doc ID: d94b3ea6-c224-49be-b5b1-5d0f42ee8041
Text: # Z. Shen et al.  Layout detection [38, 22], table detection
[26], and scene text detection [4]. A generalized learning-based
framework dramatically reduces the need for the manual specification
of complicated rules, which is the status quo with traditional
methods. DL has the potential to transform DIA pipelines and benefit a
broad spectrum of ...
--------------------------------------------------------------------------------

Document

In [9]:
# one extra dep
from llama_index.core import VectorStoreIndex


In [10]:

# create an index from the parsed markdown
vectorstore = VectorStoreIndex.from_documents(documents)


In [15]:

# create a query engine for the index
query_engine = vectorstore.as_query_engine()


In [16]:
print(f"Query Engine: {query_engine}")


Query Engine: <llama_index.core.query_engine.retriever_query_engine.RetrieverQueryEngine object at 0x000001E2A57580A0>


In [49]:

# query the engine
query = "What can you do in the Bay of Fundy?"
response = query_engine.query(query)
print(response)

You can experience the highest tides in the world in the Bay of Fundy.


In [53]:

# query the engine
query = "tell me structure of the core layoutparser library?"
response = query_engine.query(query)
print(response)

The core LayoutParser library consists of five components: 1) layout detection models, 2) stored layout information, 3) OCR module, 4) efficient data annotation, and 5) customized model training.


In [20]:

# query the engine
query = "tell me about dataset PubLayNet"
response = query_engine.query(query)
print(response)

The PubLayNet dataset focuses on layouts of modern scientific documents and is used for training layout detection models in the LayoutParser model zoo. The dataset includes models trained with different sizes for varying needs, with the base model using the ResNet 50 backbone and the large model using the ResNet 101 backbone. The dataset is one of several datasets covered by LayoutParser, providing pre-trained model weights for layout detection tasks.


In [18]:
# LLM

from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o-mini")

In [78]:
from llama_index.core.langchain_helpers.agents import IndexToolConfig, LlamaIndexTool

# Configure the tool
tool_config = IndexToolConfig(
    query_engine=query_engine,
    name="LlamaIndex Tool",
    description="Tool for querying LlamaIndex vector store",
    tool_kwargs={"return_direct": True},
)

# Create the LangChain tool
llama_index_tool = LlamaIndexTool.from_tool_config(tool_config)

In [92]:
llama_index_tool

LlamaIndexTool(name='LlamaIndex Tool', description='Tool for querying LlamaIndex vector store', return_direct=True, query_engine=<llama_index.core.query_engine.retriever_query_engine.RetrieverQueryEngine object at 0x000001E2A57580A0>)

In [79]:
from langchain.agents import initialize_agent, AgentType

# Initialize your agent with the LlamaIndex tool
agent = initialize_agent(
    tools=[llama_index_tool],
    llm=llm,  # Specify your LLM here
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

# Now you can use the agent to respond to queries
response = agent.run("tell me structure of the core layoutparser library?")
print(response)



> Entering new AgentExecutor chain...
To answer the question about the structure of the core LayoutParser library, I need to retrieve information about its organization, components, and any relevant modules or classes that are integral to its functionality. I will use the LlamaIndex Tool to query the relevant information from the vector store.

Action: LlamaIndex Tool  
Action Input: "structure of the core LayoutParser library"

Observation: The core LayoutParser library consists of five components that provide a simple interface with comprehensive functionalities. These components include layout detection models for detecting layouts with minimal code, carefully engineered data structures for storing layout information, a set of data structures and operations for efficient processing and manipulation of layout elements, support for training customized layout models, and a community platform for sharing pretrained models and document processing pipelines specific to Document Image An

In [89]:
tools = [llama_index_tool]

In [90]:
tools

[LlamaIndexTool(name='LlamaIndex Tool', description='Tool for querying LlamaIndex vector store', return_direct=True, query_engine=<llama_index.core.query_engine.retriever_query_engine.RetrieverQueryEngine object at 0x000001E2A57580A0>)]

In [98]:
from langchain.tools.retriever import create_retriever_tool

tool = create_retriever_tool(
    retriever,
    "blog_post_retriever",
    "Searches and returns excerpts from the Autonomous Agents blog post.",
)
tools = [tool]

In [99]:
tools

[Tool(name='blog_post_retriever', description='Searches and returns excerpts from the Autonomous Agents blog post.', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x000001E29F101EA0>, retriever=<llama_index.core.indices.list.retrievers.SummaryIndexLLMRetriever object at 0x000001E2A6290F70>, document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables={}, template='{page_content}'), document_separator='\n\n'), coroutine=functools.partial(<function _aget_relevant_documents at 0x000001E2A1F21A20>, retriever=<llama_index.core.indices.list.retrievers.SummaryIndexLLMRetriever object at 0x000001E2A6290F70>, document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables={}, template='{page_content}'), document_separator='\n\n'))]

In [100]:
from langgraph.prebuilt import create_react_agent

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

agent_executor = create_react_agent(llm, tools)

In [102]:
from langchain_core.messages import HumanMessage

query = "tell me structure of the core layoutparser library??"

for s in agent_executor.stream(
    {"messages": [HumanMessage(content=query)]},
):
    print(s)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_ayODoibfhScX1r0b3nToAFW8', 'function': {'arguments': '{"query":"core layoutparser library structure"}', 'name': 'blog_post_retriever'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 70, 'total_tokens': 91, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_8bfc6a7dc2', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-db866f61-cec6-426a-8b55-a73038c6bf57-0', tool_calls=[{'name': 'blog_post_retriever', 'args': {'query': 'core layoutparser library structure'}, 'id': 'call_ayODoibfhScX1r0b3nToAFW8', 'type': 'tool_call'}], usage_metadata={'input_tokens': 70, 'output_tokens': 21, 'total_tokens': 91, 'input_token_details': {'cache_read': 0}, 'output_token_details':

In [43]:
# Retriever

import bs4
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.retrievers.llama_index import LlamaIndexRetriever
# retriever = LlamaIndexRetriever(query_engine=query_engine)

retriever=vectorstore.as_retriever()

In [46]:
retriever = LlamaIndexRetriever(query_engine=vectorstore)

In [45]:
retriever

In [47]:
retriever

LlamaIndexRetriever(query_kwargs={})

In [35]:
print(query_engine)  # This should not be None


<coroutine object BaseRetriever.ainvoke at 0x000001E2A61F8890>

In [36]:
query = "What can you do in the Bay of Fundy?"
response = query_engine.query(query)
print(response)  # This should work without errors


You can experience the highest tides in the world in the Bay of Fundy.


In [48]:
response = retriever.invoke("What is the capital of France?")


AttributeError: 'NoneType' object has no attribute 'query'

In [65]:
# llama_index.core.indices.list.retrievers.ListIndexLLMRetriever

from llama_index.core.indices.list.retrievers import ListIndexLLMRetriever



In [69]:
# Create a retriever from the index
retriever = ListIndexLLMRetriever(vectorstore)

In [70]:
# Prompt

from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

In [72]:
history_aware_retriever_chain = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

TypeError: unsupported operand type(s) for |: 'function' and 'SummaryIndexLLMRetriever'

In [38]:
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

In [39]:
rag_chain = create_retrieval_chain(history_aware_retriever_chain, question_answer_chain)

In [40]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


In [41]:
# Function to generate a unique session ID
import uuid

def generate_session_id() -> str:
    return str(uuid.uuid4())

In [42]:
conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [43]:
# Generating a dynamic session ID
session_id_1 = generate_session_id()

In [ ]:
session_id_1

In [ ]:
try:
    response = conversational_rag_chain.invoke(
        {"input": "tell me about dataset PubLayNet"},
        config={
            "configurable": {"session_id": session_id_1}
        },
    )
    print(response)
except Exception as e:
    print(f"Error during invocation: {e}")

In [ ]:
from langchain.tools.retriever import create_retriever_tool

tool = create_retriever_tool(
    llamaindex_to_langchain_converted_tools,
    "blog_post_retriever",
    "Searches and returns excerpts from the Autonomous Agents blog post.",
)
tools = [tool]

In [ ]:
from langgraph.prebuilt import create_react_agent

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

agent_executor = create_react_agent(llm, tools)

In [ ]:
from langchain_core.messages import HumanMessage

query = "tell me about Related Work "

for s in agent_executor.stream(
    {"messages": [HumanMessage(content=query)]},
):
    print(s)
    print("----")